In [0]:
from google.colab import drive
drive.mount('/content/drive')
root_path = 'drive/My Drive/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!unzip "/content/drive/My Drive/project2_database.zip"

In [0]:
!pip install moviepy

In [0]:
!pip install soundfile

In [0]:
!pip install ffmpeg

  Stored in directory: /root/.cache/pip/wheels/b6/68/c3/a05a35f647ba871e5572b9bbfc0b95fd1c6637a2219f959e7a
Successfully built ffmpeg


In [0]:
!pip install pydub

In [0]:
!pip install Signal_Analysis

     |████████████████████████████████| 378kB 2.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/50/06/bb/04aa9ef50b93b5961b9817600ca1ff379f7091e63e09831655
  Stored in directory: /root/.cache/pip/wheels/6d/52/9c/94cff100c9dd4ec0c72762947b8d5da6f6c0762cd5312b04ec
Successfully built Signal-Analysis peakutils


In [0]:
!pip install python_speech_features

  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [0]:
import Signal_Analysis
from Signal_Analysis import features
from Signal_Analysis.features import signal
import numpy as np
import scipy.io.wavfile as wavfile
import librosa
import soundfile as sf
import warnings
warnings.filterwarnings("ignore")
import pickle 
import os,sys
import sklearn.mixture
from sklearn import preprocessing
import python_speech_features as mfcc
import scipy
from scipy.fftpack import dct
#import ffmpeg
#import pydub
#from pydub import AudioSegment
#import moviepy.editor as mp

In [0]:
def extract_mfcc(audio,rate):
    mfcc_feature = mfcc.mfcc(audio,rate, 0.025, 0.01,20,nfft = 1200, appendEnergy = True) 
    #mfcc_feature = preprocessing.scale(mfcc_feature)
    mfcc_feature=np.mean(mfcc_feature,axis=0)
    return mfcc_feature
#def extract_mfcc(audio,sr):
  #return np.transpose(librosa.feature.mfcc(audio, sr, n_mfcc=20))
  #mfccs = librosa.feature.mfcc(audio, sr, n_mfcc=40) 
  #return mfccs
#  mfcc = np.mean(librosa.feature.mfcc(audio,sr,n_mfcc=20).T,axis=0)
#  return mfcc


#def extract_dmfcc(audio,sr):
   
#    mfcc = np.transpose(librosa.feature.mfcc(audio, sr, n_mfcc=20))
    #mfcc = scipy.fftpack.dct(mfcc)
#    delta = np.mean(librosa.feature.delta(mfcc),axis=0)
#    return delta

  
def extract_dmfcc(audio,rate):
    mfcc_feature = mfcc.mfcc(audio,rate, 0.025, 0.01,20,nfft = 1200, appendEnergy = True)
    dmfcc_feature=librosa.feature.delta(mfcc_feature)
    #dmfcc_feature = preprocessing.scale(dmfcc_feature)
    dmfcc_feature=np.mean(dmfcc_feature,axis=0)
    return dmfcc_feature
  
def extract_log(audio,rate):
    a= mfcc.logfbank(audio,rate,0.025,0.01,20,nfft=1200)
    #a=preprocessing.scale(a)
    a=np.mean(a,axis=0)
    return a
  
#def extract_filt(audio,rate):
#  b=np.mean( preprocessing.normalize(mfcc.fbank(audio,rate,0.025,0.01,20,nfft=1200), axis = 0))
#    b=preprocessing.scale(b)
#    b=dct(b.transpose()).transpose()
  #b=np.mean(b,axis = 0)
#  return b
  
#def extract_chroma(audio,sr):
  
def extract_spec_cent(audio,sr):
  a=np.mean(librosa.feature.spectral_centroid(audio,sr))
  return np.array([a*2/sr])
                                                            
def extract_flatness(audio,sr):       #returns value in [0,1] per frame
    return np.array([np.mean(librosa.feature.spectral_flatness(audio))])
  
#def extract_pitch(audio,sr):     #returns pitch per audio in Hz
#    pitch = Signal_Analysis.features.signal.get_F_0(audio,sr)
#    return np.array([pitch[0]])
  
def extract_HNR(audio,sr):   #returns HNR per audio
    hnr = Signal_Analysis.features.signal.get_HNR(audio,sr)
    return np.array([hnr])
  
#def extract_tonnetz(audio,sr):
#  return np.mean(np.transpose(librosa.feature.tonnetz(audio,sr)),axis = 0)
  
def extract_srolloff(audio,sr):   #returns per frame in Hz
    return np.array([np.mean(librosa.feature.spectral_rolloff(audio,sr))])
  

def extract_zcr(audio,sr):
  return np.array([np.mean(librosa.feature.zero_crossing_rate(audio))])

def extract_chroma(audio,sr):
  chroma = np.mean(np.transpose(librosa.feature.chroma_stft(audio,sr)),axis = 0)
  #chroma = scipy.fftpack.dct(chroma)
  #chroma = np.mean(chroma,axis = 0)
  return np.array(chroma)

def extract_mel(audio,sr):
  return np.mean(np.transpose(librosa.feature.melspectrogram(audio, sr)),axis=0)

def extract_lfcc(signal,sample_rate):
    emphasized_signal = signal #np.append(signal[0], signal[1:] - 0.97 * signal[:-1])
    frame_length, frame_step = 0.025 * sample_rate, 0.01 * sample_rate  # Convert from seconds to samples
    signal_length = len(emphasized_signal)
    frame_length = int(round(frame_length))
    frame_step = int(round(frame_step))
    num_frames = int(np.ceil(float(np.abs(signal_length - frame_length)) / frame_step))  # Make sure that we have at least 1 frame

    pad_signal_length = num_frames * frame_step + frame_length
    z = np.zeros((pad_signal_length - signal_length))
    pad_signal = np.append(emphasized_signal, z) # Pad Signal to make sure that all frames have equal number of samples without truncating any samples from the original signal

    indices = np.tile(np.arange(0, frame_length), (num_frames, 1)) + np.tile(np.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
    frames = pad_signal[indices.astype(np.int32, copy=False)]
    
    frames *= np.hamming(frame_length)
    
    NFFT=512
    mag_frames = np.absolute(np.fft.rfft(frames, NFFT))  # Magnitude of the FFT
    pow_frames = ((1.0 / NFFT) * ((mag_frames) ** 2))  # Power Spectrum
    
    nfilt=40
    low_freq_mel = 0
    high_freq_mel = (sample_rate / 2)   
    hz_points = np.linspace(low_freq_mel, high_freq_mel, nfilt + 2)  # Equally spaced in hz scale

    bin = np.floor((NFFT + 1) * hz_points / sample_rate)

    fbank = np.zeros((nfilt, int(np.floor(NFFT / 2 + 1))))
    for m in range(1, nfilt + 1):
        f_m_minus = int(bin[m - 1])   # left
        f_m = int(bin[m])             # center
        f_m_plus = int(bin[m + 1])    # right

        for k in range(f_m_minus, f_m):
            fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
        for k in range(f_m, f_m_plus):
            fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])
    filter_banks = np.dot(pow_frames, fbank.T)
    filter_banks = np.where(filter_banks == 0, np.finfo(float).eps, filter_banks)  # Numerical Stability
    filter_banks = 20 * np.log10(filter_banks)  # dB
    
    num_ceps = 20
    lfcc = dct(filter_banks, type=2, axis=1, norm='ortho')[:, 1 : (num_ceps + 1)]
    
    cep_lifter=22
    (nframes, ncoeff) = lfcc.shape
    n = np.arange(ncoeff)
    lift = 1 + (cep_lifter / 2) * np.sin(np.pi * n / cep_lifter)
    lfcc *= lift  
    
    #lfcc -= (numpy.mean(lfcc, axis=0) + 1e-8)
    #lfcc=preprocessing.scale(lfcc)
    lfcc=np.mean(lfcc,axis=0)
    return lfcc


In [0]:
def add_noise(audio,sr):
  audio_w = audio**2
  target_snr_db = 20
  # Calculate signal power and convert to dB 
  sig_avg_watts = np.mean(audio_w)
  sig_avg_db = 10 * np.log10(sig_avg_watts)
    # Calculate noise according to [2] then convert to watts
  noise_avg_db = sig_avg_db - target_snr_db
  noise_avg_watts = 10 ** (noise_avg_db / 10)
    # Generate an sample of white noise
  mean_noise = 0
  noise_volts = np.random.normal(mean_noise, np.sqrt(noise_avg_watts), len(audio_w))
    # Noise up the original signal
  y_volts = audio + noise_volts
  return y_volts

#Extracting audio from video 

In [0]:
emo = ['anger','disgust','fear','happiness','sadness','surprise']
ind = ['an','di','fe','ha','sa','su']
count = 0
dest = "drive/My Drive/Enterface/"
y = []
f  = []

for i in range(1,45):
  for j in emo:
    for k in ind:
      for l in range(1,6):
        file = "enterface database/subject "+str(i)+"/"+j+"/"+"sentence "+str(l)+"/"+"s"+str(i)+"_"+k+"_"+str(l)+".avi"
        if os.path.isfile(file):
          clip = mp.VideoFileClip(file)
          clip.audio.write_audiofile(dest+"s"+str(i)+"_"+k+"_"+str(l)+".wav")
#audio,sr = sf.read(audioclip)
          audio,sr = sf.read(dest+"s"+str(i)+"_"+k+"_"+str(l)+".wav")
          sound = AudioSegment.from_wav(dest+"s"+str(i)+"_"+k+"_"+str(l)+".wav")
          sound = sound.set_channels(1)
          sound.export(dest+"s"+str(i)+"_"+k+"_"+str(l)+".wav", format="wav")
          audio,sr = sf.read(dest+"s"+str(i)+"_"+k+"_"+str(l)+".wav")
          count+=1
print(count)

In [0]:
y = []
f  = []
count = 0
source = "drive/My Drive/Enterface/"
ind = ['an','di','fe','ha','sa','su']
path = os.listdir(source)
for file in path:
    audio,sr=sf.read(source+file)
    lens = len(audio)/sr
    if(not(lens>=5)):
      aud=  []
      for z in range(len(audio)-1):
        aud.append(audio[z]- 0.97*audio[z-1])
      aud = np.array(aud)
     
      l1=[]
      l2 = []
      l1=extract_mfcc(audio,sr).tolist()
      l1.extend(extract_dmfcc(audio,sr).tolist())
      l1.extend(extract_log(audio,sr).tolist())
      l1.extend(extract_HNR(audio,sr).tolist())
      l1.extend(extract_lfcc(audio,sr).tolist())
                          #l1.extend(extract_filt(aud,sr).tolist())
                       
      l1.extend(extract_spec_cent(audio,sr).tolist())
      l1.extend(extract_srolloff(audio,sr).tolist())
      l1.extend(extract_chroma(audio,sr).tolist())
      l1.extend(extract_flatness(audio,sr).tolist())
      l1.extend(extract_zcr(audio,sr).tolist())
      l1.extend(extract_mel(audio,sr).tolist())
                          
      
      f.append(l1)
      
      y.append(np.array(ind.index(file.split('_')[1]))) 
      count+=1
      print(count)
print(len(y)) 
print(len(f))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [0]:
x=np.expand_dims(np.array(f),axis=2)
print(x.shape)
y=np.array(y)
print(y.shape)

(1209, 225, 1)
(1209,)


In [0]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,shuffle = True)

In [0]:
from tensorflow.keras import backend as K
K.clear_session()

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Dropout,LSTM,Masking,CuDNNLSTM,Input,concatenate,Conv1D, MaxPooling1D,Flatten,Activation, AveragePooling1D, BatchNormalization
from tensorflow.keras import regularizers

In [0]:
x1 = Input(shape = (225,1))
y1 = Conv1D(100,5, padding = 'same', activation = 'relu')(x1)
y1 = BatchNormalization(axis=1)(y1)
y1 = Dropout(0.4)(y1)

y2 = Conv1D(32,5, padding = 'same', activation = 'relu')(x1)
y2 = BatchNormalization(axis=1)(y2)
y2 = Dropout(0.4)(y2)

y1 = AveragePooling1D(pool_size=(4))(y1)
y1 = Flatten()(y1)
y2 = AveragePooling1D(pool_size=(4))(y2)

y3 = Conv1D(250,5, padding = 'same',activation = 'relu')(y2)
y3 = BatchNormalization(axis=1)(y3)
y3 = Dropout(0.4)(y3)
y3 = AveragePooling1D(pool_size = (4))(y3)
y3 = Flatten()(y3)

merged_vector = concatenate([y1,y3])

a1 = Dense(1024, activation = 'relu')(merged_vector)
a1 = Dropout(0.8)(a1)
b1 = Dense(512,activation='relu')(a1)
b1 = Dropout(0.3)(b1)
c1 = Dense(128,activation='relu')(b1)
c1 = Dropout(0.0)(c1)
d1 = Dense(64,activation = 'relu')(c1)
d1 = Dropout(0.0)(d1)
out = Dense(6,activation='softmax')(d1)
model=Model(inputs=x1 , outputs=out)


W0702 05:45:44.782817 140188086884224 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0702 05:45:45.419345 140188086884224 nn_ops.py:4224] Large dropout rate: 0.8 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


In [0]:
x1 = Input(shape = (225,1))

y1 = Conv1D(64,11, padding = 'same', activation = 'relu')(x1)
y1 = BatchNormalization(axis=1)(y1)
y1 = Dropout(0.4)(y1)
y1 = AveragePooling1D(pool_size=(4))(y1)
y1 = Flatten()(y1)

d1 = Dense(64, activation = 'relu')(y1)
d1 = Dropout(0.2)(d1)

y2 = Conv1D(32,11, padding = 'same', activation = 'relu')(x1)
y2 = BatchNormalization(axis=1)(y2)
y2 = Dropout(0.2)(y2)
y2 = MaxPooling1D(pool_size=(4))(y2)

y3 = Conv1D(64,11, padding = 'same', activation = 'relu')(y2)
y3 = Dropout(0.2)(y3)
y3 = Flatten()(y3)

d2 = Dense(512, activation = 'relu')(y3)
d2 = Dropout(0.4)(d2)

d3 = Dense(64, activation = 'relu')(d2)
d3 = Dropout(0.2)(d3)

merged_vector = concatenate([d1,d3])

out = Dense(6,activation='softmax')(merged_vector)
model=Model(inputs=x1 , outputs=out)


In [0]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 225, 1)]     0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 225, 32)      192         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 225, 32)      900         conv1d_1[0][0]                   
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 225, 32)      0           batch_normalization_1[0][0]      
______________________________________________________________________________________________

In [0]:
opt = tf.keras.optimizers.Adam(lr=1e-3 , decay= 1e-4,clipnorm=1.0,clipvalue=0.5)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [0]:
model.fit(x_train,y_train, epochs = 1000, validation_data = (x_test, y_test))

Train on 967 samples, validate on 242 samples
Epoch 1/1000
967/967 [==============================] - 1s 558us/sample - loss: 0.0366 - acc: 0.9866 - val_loss: 2.4753 - val_acc: 0.5909
Epoch 2/1000
967/967 [==============================] - 1s 618us/sample - loss: 0.0576 - acc: 0.9824 - val_loss: 2.5473 - val_acc: 0.5868
Epoch 3/1000
967/967 [==============================] - 1s 609us/sample - loss: 0.0500 - acc: 0.9876 - val_loss: 2.5491 - val_acc: 0.5702
Epoch 4/1000
967/967 [==============================] - 1s 612us/sample - loss: 0.0405 - acc: 0.9855 - val_loss: 2.5392 - val_acc: 0.5744
Epoch 5/1000
967/967 [==============================] - 1s 572us/sample - loss: 0.0587 - acc: 0.9845 - val_loss: 2.5223 - val_acc: 0.6033
Epoch 6/1000
967/967 [==============================] - 0s 511us/sample - loss: 0.0632 - acc: 0.9835 - val_loss: 2.5652 - val_acc: 0.5702
Epoch 7/1000
967/967 [==============================] - 0s 492us/sample - loss: 0.0400 - acc: 0.9866 - val_loss: 2.5466 - val_